![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\features.txt"
filename_labels = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\activity_labels.txt"

filename_subtrain = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\train\subject_train.txt"
filename_xtrain = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\train\X_train.txt"
filename_ytrain = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\train\y_train.txt"

filename_subtest = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\test\subject_test.txt"
ffilename_xtest = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\test\X_test.txt"
filename_ytest = r"C:\Users\sann_\Documentos\curso\Projetos_CD\Projecto atividade humana\dados\UCI HAR Dataset\test\y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze("columns")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [2]:
%%time

clf = DecisionTreeClassifier(random_state=123, ccp_alpha=0.001).fit(X_train, y_train)

train_scores = clf.score(X_train, y_train)
test_scores = clf.score(X_test, y_test)

print(f'Acurácia da árvore na base de treino:    {train_scores*100:.1f}')
print(f'Acurácia da árvore na base de teste:     {test_scores*100:.1f}')

Acurácia da árvore na base de treino:    97.6
Acurácia da árvore na base de teste:     88.2
CPU times: total: 3.92 s
Wall time: 3.94 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [9]:
%%time

import warnings
warnings.filterwarnings("ignore")

n=1

# aplicando PCA
prcomp = PCA(n_components=n).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

colunas = ['cp'+str(x+1) for x in list(range(n))]

# treinando modelo de árvore de decisão

clf = DecisionTreeClassifier(random_state=123, ccp_alpha=0.001).fit(pc_treino, y_train)

# calculando acurácia
train_scores = clf.score(pc_treino, y_train)
test_scores = clf.score(pc_teste, y_test)

print(f'Acurácia da árvore na base de treino:    {train_scores*100:.1f}')
print(f'Acurácia da árvore na base de teste:     {test_scores*100:.1f}')


Acurácia da árvore na base de treino:    50.0
Acurácia da árvore na base de teste:     45.7
CPU times: total: 531 ms
Wall time: 161 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [4]:
%%time

n_list = [1, 2, 5, 10, 50]

for n in n_list:
    # aplicando PCA
    prcomp = PCA(n_components=n).fit(X_train)
    pc_treino = prcomp.transform(X_train)
    pc_teste = prcomp.transform(X_test)
    colunas = ['cp'+str(x+1) for x in list(range(n))]
    pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    pc_test = pd.DataFrame(pc_teste[:,:n], columns = colunas)

    # treinando modelo de árvore de decisão
    clf = DecisionTreeClassifier(random_state=123, ccp_alpha=0.001)
    clf.fit(pc_train, y_train)

    # calculando acurácia na base de treino
    y_pred_train = clf.predict(pc_train)
    acc_train = accuracy_score(y_train, y_pred_train)

    # calculando acurácia na base de teste
    y_pred_test = clf.predict(pc_test)
    acc_teste = accuracy_score(y_test, y_pred_test)

    # imprimindo resultados
    print(f'Número de componentes principais: {n}')
    print(f'Acurácia na base de treino:       {acc_train*100:.1f}')
    print(f'Acurácia na base de teste:        {acc_teste*100:.1f}')
    print('\n')


Número de componentes principais: 1
Acurácia na base de treino:       50.0
Acurácia na base de teste:        45.7


Número de componentes principais: 2
Acurácia na base de treino:       61.3
Acurácia na base de teste:        58.5


Número de componentes principais: 5
Acurácia na base de treino:       84.6
Acurácia na base de teste:        78.9


Número de componentes principais: 10
Acurácia na base de treino:       89.1
Acurácia na base de teste:        82.2


Número de componentes principais: 50
Acurácia na base de treino:       91.9
Acurácia na base de teste:        82.2


CPU times: total: 4.62 s
Wall time: 1.4 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

   A acurácia aumenta quanto mais componentes explicativos utilizamos no nosso modelo, porém o ganho em acurácia a partir de certo número de componentes não aumenta tanto, no modelo completo utilizando as 60 variáveis explicativas nossa acurácia na base de teste foi de: 88.2, não muito diferente do modelo treinado com 10 componentes que teve acurácia de 82,2. Devemos comparar a perda de acurácia versus o peso computacional do modelo em casos de maior escalabilidade e verificar qual escolha é mais viável.  